# NCPT Paperspace One-click (v0.2)

### Last updated: 21 Sept 2023

Check for updates [here](https://link-will-be-here-soon.com) (eventually)


In [1]:
from ipywidgets import Checkbox, Text
from IPython.display import display
from pathlib import Path
from subprocess import check_output, check_call


style = {"description_width": "25%"}
layout = {"width": "50%"}

persistence_control = Checkbox(
    value=False,
    description="Save outputs to persistent storage (max. 5GB for free, 15GB for pro)",
    layout=layout,
)
force_cache_control = Checkbox(
    value=False,
    description="Force use of cached dependencies (faster, may break things)",
    layout=layout,
)
extn_update_control = Checkbox(value=False, description="Update all extensions on launch", layout=layout)

# textboxes
hf_token = Text(value="", description="HuggingFace token (optional)", style=style, layout=layout)
commandline_arg_control = Text(
    value="--xformers", description="Extra commandline arguments", style=style, layout=layout
)

# show them
display(persistence_control, force_cache_control, extn_update_control, hf_token, commandline_arg_control)


Checkbox(value=False, description='Save outputs to persistent storage (max. 5GB for free, 15GB for pro)', layo…

Checkbox(value=False, description='Force use of cached dependencies (faster, may break things)', layout=Layout…

Checkbox(value=False, description='Update all extensions on launch', layout=Layout(width='50%'))

Text(value='', description='HuggingFace token (optional)', layout=Layout(width='50%'), style=TextStyle(descrip…

Text(value='--xformers', description='Extra commandline arguments', layout=Layout(width='50%'), style=TextStyl…

In [ ]:
# ---------[ No need to change anything in this block! ]---------

import shutil
from urllib.parse import urlparse
from os import getenv, environ

# fixed (Always present) command line args, needed for this to work at all
static_args = "--enable-insecure-extension-access --disable-safe-unpickle --theme dark --listen --port 6006"

# options class for easy access
class CurrentOptions:
    # fixed (Always present) command line args, needed for this to work at all
    static_args = (
        "--enable-insecure-extension-access --disable-safe-unpickle --theme dark --listen --port 6006"
    )

    @property
    def output_to_persistent(self) -> bool:
        return persistence_control.value

    @property
    def force_cache_deps(self) -> bool:
        return force_cache_control.value

    @property
    def update_all_extensions(self) -> bool:
        return extn_update_control.value

    @property
    def optional_huggingface_token(self) -> str:
        return hf_token.value

    @property
    def commandline_arguments(self) -> str:
        arg_list = [self.static_args]
        if hf_token.value != "":
            arg_list.extend(["--hf-token-out", hf_token.value])
        if extn_update_control.value is True:
            arg_list.append("--update-all-extensions")
        if commandline_arg_control.value != "":
            arg_list.extend([x for x in commandline_arg_control.value.split(" ") if x != ""])
        return " ".join(arg_list)

# spawn the boi
options = CurrentOptions()

# paperspace-provided directories
notebook_dir = Path("/notebooks")
storage_dir = Path("/storage")
# local-only semi-persistent storage
content_dir = Path("/content")
# where we put deps
dep_cache_dir = storage_dir.joinpath("a1111_dependencies")
# where models will go
models_dir = content_dir.joinpath("sdw/models")
# temp file for aria2 jobs
aria2_file = Path("/tmp/aria2-in.txt")


# set up content dir symlink
content_dir.mkdir(exist_ok=True)
content_symlink = content_dir.joinpath("content")
if not all(
    (content_symlink.exists(), content_symlink.is_symlink(), content_symlink.resolve() == content_dir)
):
    # recreate the symlink
    shutil.rmtree(content_symlink, ignore_errors=True)
    content_symlink.symlink_to(content_dir, target_is_directory=True)

# download repo if needed
config_json = content_dir.joinpath("sdw/config.json")
if not config_json.exists():
    # repo isn't here so LET'S GOOOOOOOOOO
    aria2_task = "\n".join(
        [
            "https://huggingface.co/NoCrypt/fast-repo/resolve/main/repo.tar.lz4",
            "\tout=repo.tar.lz4",
            "https://huggingface.co/NoCrypt/fast-repo/resolve/main/cache.tar.lz4",
            "\tout=cache.tar.lz4",
        ]
    )
    aria2_file.write_text(aria2_task)
    display("Downloading repo and cache. This may take a while...")
    check_call(
        ["aria2c", "--input-file=/tmp/aria2-in.txt", "-j4", "-x16", "-s16", "-k1M", "-c"],
        cwd=content_dir,
    )
    aria2_file.unlink()

    display("Extracting repo...")
    check_call(["tar", "-xI", "lz4", "-f", "repo.tar.lz4", "--directory=/"], cwd=content_dir)
    content_dir.joinpath("repo.tar.lz4").unlink()

    display("Extracting cache...")
    check_call(["tar", "-xI", "lz4", "-f", "cache.tar.lz4", "--directory=/"], cwd=content_dir)
    content_dir.joinpath("cache.tar.lz4").unlink()

# set up dependency cache
if dep_cache_dir.exists() is False or options.force_cache_deps:
    if options.force_cache_deps and dep_cache_dir.exists():
        display("Forcing removal of dependency cache!")
        shutil.rmtree(dep_cache_dir)
    display("Caching dependencies. This may take a while...")
    dep_cache_dir.mkdir(exist_ok=True)
    check_call(
        ["bash", "./webui.sh", "-f", options.commandline_arguments, "--exit"], cwd=content_dir.joinpath("sdw")
    )

    # recursively copy all files in /usr/local/lib/python3.10/site-packages/ to /storage/a1111_dependencies/ (except for .pyc files)
    if Path("/usr/local/lib/python3.10/site-packages/").exists():
        display("Copying dependencies to persistent cache...")
        shutil.copytree(
        "/usr/local/lib/python3.10/site-packages/",
        "/storage/a1111_dependencies/",
        symlinks=True,
        ignore=shutil.ignore_patterns("*.pyc"),
        dirs_exist_ok=True,
    )

# func to acquire checkpoints with
def acquire_checkpoints(checkpoint_urls: list[str], subdir:str = "Stable-diffusion") -> bool:
    aria2_task_lines = []
    for url in checkpoint_urls:
        filename = urlparse(url).path.split("/")[-1]
        if not models_dir.joinpath(subdir, filename).exists():
            aria2_task_lines.extend([url, f"\tout={filename}"])
    if len(aria2_task_lines) == 0:
        display(f"All {subdir} checkpoints already downloaded, skipping...")
        return
    aria2_task = "\n".join(aria2_task_lines)
    aria2_file.write_text(aria2_task)
    display(f"Downloading {len(checkpoint_urls)} checkpoints, this may take a while...")
    check_call(
        ["aria2c", "--input-file=/tmp/aria2-in.txt", "-j4", "-x16", "-s16", "-k1M", "-c"],
        cwd=models_dir.joinpath(subdir),
    )
    aria2_file.unlink()
    return True

# what ones to load
model_urls = [
    "https://huggingface.co/NoCrypt/expermental_models/resolve/main/shux4.fp16.safetensors",
    "https://huggingface.co/NoCrypt/expermental_models/resolve/main/smoothshux4d-half.safetensors",
    "https://huggingface.co/NoCrypt/expermental_models/resolve/main/exs4.safetensors",
]
vae_urls = [
    'https://huggingface.co/NoCrypt/resources/resolve/main/VAE/any.vae.safetensors',
    'https://huggingface.co/NoCrypt/resources/resolve/main/VAE/blessed2.vae.safetensors',
    'https://huggingface.co/NoCrypt/resources/resolve/main/VAE/wd-blessed09.vae.safetensors',
    'https://huggingface.co/NoCrypt/resources/resolve/main/VAE/wd.vae.safetensors',
]

# actually load them
acquire_checkpoints(model_urls, "Stable-diffusion")
acquire_checkpoints(vae_urls, "VAE")

# start. the thing
environ['COMMANDLINE_ARGS'] = options.commandline_arguments
environ['REQS_FILE'] = "requirements_versions.txt"

display("Starting 'tensorboard'...")
display(f"Access via this URL: https://tensorboard-{getenv('PAPERSPACE_FQDN')}")

cmdline_arg_str = options.commandline_arguments

webui_user = f"""#!/bin/bash
venv_dir='-'
"""
content_dir.joinpath("sdw/webui-user.sh").write_text(webui_user)

%cd /content/sdw
check_call(["/usr/bin/env", "bash", "./webui.sh", "-f"], cwd=content_dir.joinpath("sdw"))